In [42]:
import os
import re
import jieba
import pickle
import math
from collections import defaultdict

In [4]:
def read_text(file_path):
    with open(file_path,'r',encoding='gb18030') as f:
        text = f.read().splitlines()
    flag = 0
    content=[]
    for item in text:
        if item == '<TEXT>':
            flag =1
            continue
        elif item == '</TEXT>':
            flag =0
            continue
        else:
            if flag ==1 :
                content.append(item)
    punc = '~`!#$%^&*()_+-=|\';":/.,?><~·！@#￥%……&*（）——：’；、。，“”？》《{}【】'
    content = re.sub('[%s]'%punc, "",' '.join(content))
    return content

In [5]:
# file_path = '/Users/bbbblink.q/1_Python/spam/email/corpus/train/Data/001/001'
# read_text(file_path)

In [6]:
def cut_word(text):
    jieba_list = jieba.cut(text, cut_all=False)
    jieba_txt = ' ' .join(jieba_list)
    jieba_txt =jieba_txt.split( )
    return jieba_txt

In [7]:
def Counter(txt):
    counter={}
    for word in txt:
        if word not in counter:
            counter[word] = 1
    return counter

In [8]:
def merge_dict(x,y):
    for k,v in x.items():
        if k in y.keys():
            y[k]+=v
        else:
            y[k]=v

In [ ]:
#预处理训练集
spam=[]
ham=[]
i=0
index_path='/Users/bbbblink.q/1_Python/spam/email/index.txt'
with open (index_path,'r') as f:
    text = f.readlines()
    text = text[1: -5000]
for item in text:
    item = item.split()
    label = item[0]
#     print(label)
    f_path = item[1]
    f_path = f_path.replace("..","/Users/bbbblink.q/1_Python/spam/email/corpus/train")
    if os.path.isfile(f_path):
        content1 = read_text(f_path) 
        if label == 'spam':
            spam.append(content1)
        else:
            ham.append(content1)
    else:
        print('loss file: {}'.format(f_path))
        i=i+1
        continue
print(i)

In [ ]:
#创建训练集
file = open('train_spam.pickle', 'wb')
pickle.dump(spam, file)
file.close()

file = open('train_ham.pickle', 'wb')
pickle.dump(ham, file)
file.close()

In [40]:
#计算先验概率
with open('train_spam.pickle', 'rb') as file:
    spam = pickle.load(file)
with open('train_ham.pickle','rb') as file:
    ham = pickle.load(file)

c_spam = len(spam)
c_ham = len(ham)
total = c_spam + c_ham
p_spam = c_spam / total
p_ham = c_ham / total
print(p_spam)
print(p_ham)

0.7443700953693617
0.2556299046306382


In [43]:
#构建counter_spam
counter_s = defaultdict(int)
i = 0
with open('train_spam.pickle', 'rb') as file:
    spam_text = pickle.load(file)
for item in spam_text:
    text_cut = cut_word(item)
    b=Counter(text_cut)
    merge_dict(b,counter_s)
    i = i+1
    if i % 500==0:
        print(i,end=" ")

500 1000 1500 2000 2500 3000 3500 4000 4500 5000 5500 6000 6500 7000 7500 8000 8500 9000 9500 10000 10500 11000 11500 12000 12500 13000 13500 14000 14500 15000 15500 16000 16500 17000 17500 18000 18500 19000 19500 20000 20500 21000 21500 22000 22500 23000 23500 24000 24500 25000 25500 26000 26500 27000 27500 28000 28500 29000 29500 30000 30500 31000 31500 32000 32500 33000 

In [44]:
# 将数据存入pickle
file = open('counter_spam.pickle', 'wb')
pickle.dump(counter_s, file)
file.close()

In [45]:
#构建counter_ham
counter_h = defaultdict(int)
i = 0
with open('train_ham.pickle', 'rb') as file:
    spam_text = pickle.load(file)
for item in spam_text:
    text_cut = cut_word(item)
    b=Counter(text_cut)
    merge_dict(b,counter_h)
    i = i+1
    if i % 500==0:
        print(i,end=" ")

500 1000 1500 2000 2500 3000 3500 4000 4500 5000 5500 6000 6500 7000 7500 8000 8500 9000 9500 10000 10500 11000 

In [46]:
# 将数据存入pickle
file = open('counter_ham.pickle', 'wb')
pickle.dump(counter_h, file)
file.close()

In [ ]:
# for i,item in enumerate(counter_h.items()):
#     print(item,end="")
#     if i==100:
#         break

In [10]:
#预处理测试集
gold = []
contents = []
i=0
index_path='/Users/bbbblink.q/1_Python/spam/email/index.txt'
with open (index_path,'r') as f:
    text = f.readlines()
test= text[-5000:] 
for item in test:
    item = item.split()
    label = item[0]
    f_path = item[1]
    f_path = f_path.replace("..","/Users/bbbblink.q/1_Python/spam/email/corpus/train")
    if os.path.isfile(f_path):
        text=read_text(f_path)
        text_cut = cut_word(text)
        contents.append(text_cut)
        if label == 'spam':
            gold.append(1)
        else:
            gold.append(0)
    else:
        print('loss file: {}'.format(f_path))
    i=i+1    
    if i % 500 == 0:
        print(i,end=" ")
    continue 

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/_g/7hk997tj1hq1fnspjw938hcc0000gn/T/jieba.cache
Loading model cost 0.685 seconds.
Prefix dict has been built successfully.


500 1000 1500 2000 2500 3000 3500 4000 4500 5000 

In [68]:
file = open('test.pickle', 'wb')
pickle.dump(contents, file)
file.close()

In [133]:
#开始测试正确率
def naive_bayes(content):
    result = 0
    p_x_spam = 0
    p_x_ham = 0
    for word in content:
        if remove_stop:
            if word in stop_list:
                continue
        if spam_c[word]== 0 or ham_c[word] == 0:
            continue
        p_x_spam = math.log((spam_c[word])/c_spam)
        p_x_ham = math.log((ham_c[word])/c_ham)
    p_spam_x = p_x_spam + math.log(p_spam)
    p_ham_x = p_x_ham + math.log(p_ham)
    if p_spam_x / p_ham_x < threthold:
        result = 1  
    else:
        result = 0
    return result

In [134]:
spam_c={}
ham_c={}
with open('counter_spam.pickle', 'rb') as file:
    spam_c = pickle.load(file)

with open('counter_ham.pickle','rb') as file:
    ham_c = pickle.load(file)

with open('test.pickle','rb') as file:
    contents = pickle.load(file)

#构建停用词表
stop_path1='/Users/bbbblink.q/1_Python/spam/email/corpus/stop_list/baidu_stopwords.txt'
with open(stop_path1,'r') as f:
    stop_list1 = f.read().splitlines()
stop_path2='/Users/bbbblink.q/1_Python/spam/email/corpus/stop_list/scu_stopwords.txt'
with open(stop_path2,'r') as f:
    stop_list2 = f.read().splitlines()
for item in stop_list2:
    stop_list1.append(item)
stop_list = stop_list1

In [135]:
test_result=[]
remove_stop = True
threthold = 1
for item in contents:
    test_result.append(naive_bayes(item))

In [136]:
c = 0
for i in range(len(gold)):
    if test_result[i] == gold[i]:
        c = c + 1
    else:
        continue
acc = c / len(gold)
print(acc)

0.9286
